# 1. Import Dataset

In [10]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping

# 2. Import Dataset

In [4]:
data_dir = "C:/Users/vv093/Downloads/archive (1)/train/"
X, y = [], []
for label in os.listdir(data_dir):
    folder = os.path.join(data_dir, label)
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            resized = cv2.resize(img, (48, 48))
            X.append(resized)
            y.append(label)

In [5]:
#Find the how many Train data Images is there
total_images = 0  
for emotion_folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, emotion_folder)
    if os.path.isdir(folder_path):
        image_files = [f for f in os.listdir(folder_path)
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        count = len(image_files)
        total_images += count
        print(f" Folder '{emotion_folder}': {count} images")
print(f"\n Total images in dataset: {total_images}")

 Folder 'angry': 3995 images
 Folder 'disgust': 436 images
 Folder 'fear': 4097 images
 Folder 'happy': 7215 images
 Folder 'neutral': 4965 images
 Folder 'sad': 4830 images
 Folder 'surprise': 3171 images

 Total images in dataset: 28709


# 3.Label Encoder

In [6]:
X = np.array(X).reshape(-1, 48, 48, 1) / 255.0
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y)

# 4. Split the Dataset

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 5. Built a CNN Model

In [12]:
# CNN model
model = Sequential([
    Input(shape=(48,48,1)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

## Compile the model

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 5.Train the CNN model

In [17]:
# Train
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=64)

Epoch 1/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 41s 113ms/step - accuracy: 0.6411 - loss: 0.9622 - val_accuracy: 0.5686 - val_loss: 1.1346
Epoch 2/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 41s 115ms/step - accuracy: 0.6578 - loss: 0.9186 - val_accuracy: 0.5674 - val_loss: 1.1550
Epoch 3/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 40s 112ms/step - accuracy: 0.6533 - loss: 0.9084 - val_accuracy: 0.5723 - val_loss: 1.1469
Epoch 4/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 40s 111ms/step - accuracy: 0.6663 - loss: 0.8833 - val_accuracy: 0.5751 - val_loss: 1.1526
Epoch 5/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 40s 112ms/step - accuracy: 0.6769 - loss: 0.8596 - val_accuracy: 0.5737 - val_loss: 1.1510
Epoch 6/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 41s 113ms/step - accuracy: 0.6972 - loss: 0.8285 - val_accuracy: 0.5801 - val_loss: 1.1653
Epoch 7/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 39s 107ms/step - accuracy: 0.6985 - loss: 0.8167 - val_accuracy: 0.5731 - val_loss: 1.1636
Epoch 8/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 37s 103ms/step - accuracy: 0.7052 - loss: 0

In [18]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print accuracy
print(f"\n Final Test Accuracy: {test_accuracy * 100:.2f}%")

180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.5800 - loss: 1.3247

 Final Test Accuracy: 57.56%


# 6.Save the Model

In [19]:
os.makedirs("model", exist_ok=True)
model.save("model/emotion_model.keras")
os.makedirs("Classes", exist_ok=True)
np.save("Classes/classes.npy", le.classes_)